In [1]:
import spatialite
import pandas as pd
import cv2
import numpy as n
import os

import pathlib

# TODO

Make it so annotations are dynamically positioned to remain withing the borders of the image

In [2]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

time: 319 µs (started: 2023-12-07 11:35:55 +08:00)


In [3]:
DBPATH = '700_990.db'
ORIGINAL_IMAGE_DIR = '../rawdata'
ANNOTATED_IMAGE_DIR = '../700_990'
IMAGES_WITH_VCUTS = '../700_990_images_with_vcuts'

time: 1.06 ms (started: 2023-12-07 11:35:55 +08:00)


In [4]:
def get_images():
    # create a dataframe for all images which contain boxes (detected coconut palms)
    sql = '''
    SELECT images.id, imagefile 
    FROM images, boxes 
    WHERE images.id=boxes.images_id
    GROUP BY images.id
    '''
    df_images = pd.read_sql(sql, con)
    return df_images

time: 1.44 ms (started: 2023-12-07 11:35:55 +08:00)


In [5]:
def get_boxes_and_polygons(imagefile):
    # get data for objects detected in imagefile
    # returns 2 dataframes:
    # df_boxes contains data for coconut palms
    # df_polygons contains data for v-shaped cuts

    # Get boxes in imagefile
    sql = f'''SELECT boxes.*, AsText(boxes.geometry) as wkt
    FROM images, boxes 
    WHERE images.id=boxes.images_id AND imagefile="{imagefile}";
    '''
    df_boxes = pd.read_sql(sql, con)
    
    # Get polygons which intersect the selected boxes
    box_ids = tuple(df_boxes.id)
    sql = f'''SELECT * 
    FROM intersections 
    WHERE boxes_id IN {box_ids};
    '''
    sql = sql.replace(',)', ')')
    df_intersections = pd.read_sql(sql, con)
    polygons_ids = tuple(df_intersections.polygons_id)

    sql = f'select *, AsText(geometry) as wkt from polygons where id in {polygons_ids}'
    sql = sql.replace(',)', ')')
    df_polygons = pd.read_sql(sql, con)
    
    return df_boxes, df_polygons    

time: 1.19 ms (started: 2023-12-07 11:35:55 +08:00)


In [6]:
def poly_wkt_to_rectangle(wkt_string):
    # Returns coordinates for a rectangle bounding a polygon stored as a WKT string
    # Useful for drawing rectangles on cv2 images
    s = wkt_string
    s = s.replace('POLYGON((', '').replace('))', '').replace(',', '')
    s = s.split()
    xylist = [int(i) for i in s]
    xmin = min(xylist[0::2])
    xmax = max(xylist[0::2])
    ymin = min(xylist[1::2])
    ymax = max(xylist[1::2])
    return xmin, ymin, xmax, ymax

# wkt = 'POLYGON((231 0, 231 796, 748 796, 748 0, 231 0))'
# xmin, ymin, xmax, ymax = poly_wkt_to_rectangle(wkt)
# print(xmin, ymin, xmax, ymax)

time: 2.94 ms (started: 2023-12-07 11:35:55 +08:00)


In [7]:
def draw_text(img, text, 
              font=cv2.FONT_HERSHEY_PLAIN,
              pos=(0, 0),
              font_scale=2,
              font_thickness=2,
              text_color=(0, 255, 0),
              text_color_bg=(0, 0, 0)
    ):    
    x, y = pos
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    cv2.rectangle(img, pos, (x + text_w, y + text_h), text_color_bg, -1)
    cv2.putText(img, text, (x, y + text_h + font_scale - 1), font, font_scale, text_color, font_thickness)
    return text_size

time: 2.25 ms (started: 2023-12-07 11:35:55 +08:00)


In [8]:
def annotate(imagefile):
    inputpath = f'{ORIGINAL_IMAGE_DIR}/{imagefile}'
    date = imagefile[4:12]
    hour = imagefile[13:15]
    outputdir = f'{ANNOTATED_IMAGE_DIR}/{date}/{hour}'
    pathlib.Path(outputdir).mkdir(parents=True, exist_ok=True)   # ensure that outputdir exists
    outputpath = f'{outputdir}/{imagefile}'    
    print(outputpath)
            
    df_boxes, df_polygons = get_boxes_and_polygons(imagefile)
    
    img = cv2.imread(inputpath)

    # draw boxes
    for i, r in df_boxes.iterrows():
        color = colors[r.damage_level]
        xmin, ymin, xmax, ymax = poly_wkt_to_rectangle(r.wkt)
        img = cv2.rectangle(img, (xmin,ymin), (xmax,ymax), color, 3)
        text = f'{r.id} {r.score:.4f} {r.damage_level}'
        draw_text(img, text, pos=(xmin+5,ymax-27)) 

    # draw polygons
    for i, r in df_polygons.iterrows():
        xmin, ymin, xmax, ymax = poly_wkt_to_rectangle(r.wkt)
        img = cv2.rectangle(img, (xmin,ymin), (xmax,ymax), (0,0,255), 1)
        text = f'{r.id} {r.score:.4f}'    
        draw_text(img, text, text_color=(0,0,255), pos=(xmin-100,ymax+5)) 

    cv2.imwrite(outputpath, img)  
    
# annotate('IMG_20231007_111032.jpg')

time: 2.08 ms (started: 2023-12-07 11:35:55 +08:00)


In [9]:
# def create_top100_symlinks():
#     # Get v-shaped cuts which intersect with boxes and select those with the top 100 highest scores
#     sql = '''
#     SELECT imagefile, polygons.score 
#     FROM images, polygons, intersections
#     WHERE images.id=polygons.images_id 
#       AND polygons.id=intersections.polygons_id
#     ORDER BY polygons.score DESC
#     LIMIT 100;
#     '''
#     df_top100 = pd.read_sql(sql, con)
    
#     # delete output directory, create a new one and use it as the work directory
#     os.system(f'rm -rf {TOP100POLYGONS}') 
#     os.system(f'mkdir {TOP100POLYGONS}')     
#     original_wd = os.getcwd()
#     os.chdir(TOP100POLYGONS)
    
#     for i, r in df_top100.iterrows():
#         imagefile = r.imagefile
#         command = f'ln -s {ANNOTATED_IMAGE_DIR}/{imagefile} {i:02}_{imagefile}'
#         os.system(command)
        
#     # change back to original work directory
#     os.chdir(original_wd)

time: 3.41 ms (started: 2023-12-07 11:35:55 +08:00)


In [10]:
def create_vcuts_symlinks():
    sql = '''
    select imagefile
    from images, boxes, intersections
    where 
      images.id=boxes.images_id
      and boxes.id=intersections.boxes_id
    group by imagefile;
    '''
    df = pd.read_sql(sql, con)
    
    print(df)
                     
    # delete output directory, create a new one and use it as the work directory
    os.system(f'rm -rf {IMAGES_WITH_VCUTS}') 
    os.system(f'mkdir {IMAGES_WITH_VCUTS}')     
    original_wd = os.getcwd()
    os.chdir(IMAGES_WITH_VCUTS)
    
    for i, r in df.iterrows():
        imagefile = r.imagefile
        command = f'ln -s {ANNOTATED_IMAGE_DIR}/{imagefile} {i:02}_{imagefile}'
        os.system(command)
        
    # change back to original work directory
    os.chdir(original_wd)    

time: 1.94 ms (started: 2023-12-07 11:35:55 +08:00)


In [11]:
def create_d5_images_dir():
    sql = """
    SELECT imagefile
    FROM images
    JOIN boxes ON images.id=boxes.images_id
    WHERE damage_level=5;
    """

    output_dir = '../d5_images'
    os.system(f'rm -rf {output_dir}') 
    os.system(f'mkdir {output_dir}')     

    with spatialite.connect(DBPATH) as con:
        df = pd.read_sql(sql, con)

    for i, r in df.iterrows():
        command = f'cp {ANNOTATED_IMAGE_DIR}/{r.imagefile} {output_dir}/{r.imagefile}'
        os.system(command) 

create_d5_images_dir()

time: 144 ms (started: 2023-12-07 11:35:55 +08:00)


In [12]:
# MAIN

# delete output directory and create a new one
os.system(f'rm -rf {ANNOTATED_IMAGE_DIR}') 
os.system(f'mkdir {ANNOTATED_IMAGE_DIR}')

# colors for damage classes
colors = {1:(0,255,0), 2:(0,255,255), 3:(0,165,255), 4:(0,100,255), 5:(0,0,255)}

print('---annotating images')
con = spatialite.connect(DBPATH)
df_images = get_images()
for i, r in df_images.iterrows():
    imagefile = r.imagefile
    annotate(imagefile)
    
# print('---creating symlinks for images with top 100 polygons') 
# create_top100_symlinks()

print('---creating symlinks for images with v-cuts') 
create_vcuts_symlinks()

con.close()

print('FINISHED')

---annotating images
../700_990/20231007/11/IMG_20231007_110803.jpg
../700_990/20231007/11/IMG_20231007_110821.jpg
../700_990/20231007/11/IMG_20231007_110845.jpg
../700_990/20231007/11/IMG_20231007_110846.jpg
../700_990/20231007/11/IMG_20231007_110847.jpg
../700_990/20231007/11/IMG_20231007_110848.jpg
../700_990/20231007/11/IMG_20231007_110849.jpg
../700_990/20231007/11/IMG_20231007_110850.jpg
../700_990/20231007/11/IMG_20231007_110948.jpg
../700_990/20231007/11/IMG_20231007_110949.jpg
../700_990/20231007/11/IMG_20231007_110951.jpg
../700_990/20231007/11/IMG_20231007_110952.jpg
../700_990/20231007/11/IMG_20231007_110953.jpg
../700_990/20231007/11/IMG_20231007_110954.jpg
../700_990/20231007/11/IMG_20231007_110955.jpg
../700_990/20231007/11/IMG_20231007_110956.jpg
../700_990/20231007/11/IMG_20231007_110957.jpg
../700_990/20231007/11/IMG_20231007_111001.jpg
../700_990/20231007/11/IMG_20231007_111008.jpg
../700_990/20231007/11/IMG_20231007_111013.jpg
../700_990/20231007/11/IMG_20231007_111